
[Dataset](https://drive.google.com/file/d/1wWsrN2Ep7x6lWqOXfr4rpKGYrJhWc8z7/view)

In [1]:
import os
Root = "C:\\Users\\Yogita\\Downloads\\speech-emotion-recognition-ravdess-data"
os.chdir(Root)

In [2]:
ls

 Volume in drive C is Windows-SSD
 Volume Serial Number is CCC9-3214

 Directory of C:\Users\Yogita\Downloads\speech-emotion-recognition-ravdess-data

29-04-2023  16:12    <DIR>          .
16-06-2023  11:33    <DIR>          ..
25-04-2023  21:34    <DIR>          Actor_01
25-04-2023  21:35    <DIR>          Actor_02
25-04-2023  21:35    <DIR>          Actor_03
25-04-2023  21:35    <DIR>          Actor_04
25-04-2023  21:35    <DIR>          Actor_05
25-04-2023  21:35    <DIR>          Actor_06
25-04-2023  21:35    <DIR>          Actor_07
25-04-2023  21:35    <DIR>          Actor_08
25-04-2023  21:35    <DIR>          Actor_09
25-04-2023  21:35    <DIR>          Actor_10
25-04-2023  21:35    <DIR>          Actor_11
25-04-2023  21:35    <DIR>          Actor_12
25-04-2023  21:35    <DIR>          Actor_13
25-04-2023  21:35    <DIR>          Actor_14
25-04-2023  21:35    <DIR>          Actor_15
25-04-2023  21:35    <DIR>          Actor_16
25-04-2023  21:35    <DIR>          Actor_17
25-04-2

In [3]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [4]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [5]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [6]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("C:\\Users\\Yogita\\Downloads\\speech-emotion-recognition-ravdess-data\\Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=12)

In [7]:
def extract_feature(file_name, mfcc=True, chroma=True, mel=True):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            stft=np.abs(librosa.stft(X))
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [8]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.05)

In [9]:
x_train

array([[-5.10513763e+02,  4.24358864e+01, -9.45346069e+00, ...,
         1.64437573e-04,  9.46390792e-05,  5.23333256e-05],
       [-4.56259705e+02,  3.64639740e+01, -1.66679306e+01, ...,
         9.26713750e-04,  4.61736636e-04,  3.27778573e-04],
       [-5.60618164e+02,  5.78945961e+01, -8.87167645e+00, ...,
         6.46989676e-04,  3.31015413e-04,  1.85833647e-04],
       ...,
       [-6.28208679e+02,  6.78257980e+01,  3.31187057e+00, ...,
         1.69829382e-05,  2.15309719e-05,  8.10299025e-06],
       [-6.99333679e+02,  5.65012970e+01, -4.47911210e-02, ...,
         4.29499642e-06,  3.19059973e-06,  2.37793006e-06],
       [-5.39937622e+02,  4.63176537e+01, -8.90276718e+00, ...,
         1.99300077e-04,  1.41271099e-04,  9.42719635e-05]])

In [10]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(729, 39)


In [11]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [12]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [13]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [14]:
#Predict for the test set
y_pred=model.predict(x_test)

In [15]:
y_pred

array(['calm', 'happy', 'happy', 'happy', 'calm', 'calm', 'disgust',
       'disgust', 'happy', 'happy', 'calm', 'fearful', 'fearful',
       'fearful', 'disgust', 'fearful', 'fearful', 'calm', 'fearful',
       'happy', 'calm', 'happy', 'disgust', 'happy', 'happy', 'disgust',
       'disgust', 'disgust', 'happy', 'fearful', 'happy', 'happy',
       'happy', 'calm', 'disgust', 'happy', 'calm', 'fearful', 'calm'],
      dtype='<U7')

In [16]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 76.92%


In [17]:
from sklearn.metrics import accuracy_score, f1_score

In [18]:
f1_score(y_test, y_pred,average=None)

array([0.75      , 0.8       , 0.8       , 0.72727273])

In [19]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,calm,calm
1,happy,happy
2,happy,happy
3,happy,happy
4,calm,calm
5,calm,calm
6,disgust,disgust
7,disgust,disgust
8,happy,happy
9,happy,happy


In [20]:
import pickle
# Writing different model files to file
with open( 'modelForPrediction1.sav', 'wb') as f:
    pickle.dump(model,f)

In [21]:
filename = 'modelForPrediction1.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature("C:\\Users\\Yogita\\Downloads\\speech-emotion-recognition-ravdess-data\\Actor_01\\03-01-02-01-01-01-01.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

array(['calm'], dtype='<U7')

In [22]:
feature

array([[-7.09056824e+02,  5.57343063e+01,  2.66831017e+00,
         1.63625717e+01,  3.34478068e+00, -1.12484837e+00,
        -5.81839275e+00, -8.99917793e+00, -9.00914574e+00,
         1.83856320e+00, -4.59688282e+00,  8.66339445e-01,
        -5.26519156e+00,  2.29214072e+00, -3.53222799e+00,
        -4.00518894e+00, -9.01319921e-01, -2.07280874e+00,
        -4.99126625e+00, -2.39771295e+00, -3.05734110e+00,
        -5.33976030e+00, -4.98443782e-01, -4.57491541e+00,
        -1.80997217e+00, -7.89660990e-01, -1.83776867e+00,
        -4.33098048e-01, -2.91427803e+00, -1.27407944e+00,
        -3.70224690e+00, -6.55556202e-01, -1.61118352e+00,
        -1.53617072e+00, -3.60538459e+00, -3.62369895e+00,
        -4.16981697e+00, -2.33298492e+00, -1.29951084e+00,
        -1.49847436e+00,  6.03868425e-01,  6.26373410e-01,
         6.96483552e-01,  6.88914716e-01,  6.79974377e-01,
         6.92952931e-01,  6.99749649e-01,  7.20743597e-01,
         7.34567702e-01,  7.47884452e-01,  7.74087787e-0